In [10]:
import cv2
import torch
import time
import numpy as np

In [11]:
#model_type = "DPT_Large"   #MiDas v3 - Large (Highest accuracy, slowest inference speed)
model_type = "DPT_Hybrid"  #MiDas v3 - Hybrid (Medium accuracy, medium inference speed)
#model_type = "MiDaS_small"

In [12]:
midas = torch.hub.load("intel-isl/MiDaS", model_type)

Using cache found in /home/sacchin/.cache/torch/hub/intel-isl_MiDaS_master


In [13]:
device = torch.device("cuda") 
midas.to(device)
midas.eval()

DPTDepthModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): HybridEmbed(
        (backbone): ResNetV2(
          (stem): Sequential(
            (conv): StdConv2dSame(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
            (norm): GroupNormAct(
              32, 64, eps=1e-05, affine=True
              (drop): Identity()
              (act): ReLU(inplace=True)
            )
            (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
          )
          (stages): Sequential(
            (0): ResNetStage(
              (blocks): Sequential(
                (0): Bottleneck(
                  (downsample): DownsampleConv(
                    (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                    (norm): GroupNormAct(
                      32, 256, eps=1e-05, affine=True
                      (drop): Identity()
                      (act): Identit

In [5]:
print(torch.cuda.is_available())

True


In [14]:
midas_transforms = torch.hub.load("intel-isl/MiDaS","transforms")

Using cache found in /home/sacchin/.cache/torch/hub/intel-isl_MiDaS_master


In [15]:
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

In [17]:
cap = cv2.VideoCapture(2)

while cap.isOpened():
    success, img = cap.read()
    start = time.time()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    input_batch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(prediction.unsqueeze(1), size=img.shape[:2],mode="bicubic",align_corners=False,).squeeze()

        depth_map = prediction.cpu().numpy()

        depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)

        end = time.time()
        totalTime = end - start

        fps = 1/ totalTime

        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        depth_map = (depth_map*255).astype(np.uint8)
        depth_map = cv2.applyColorMap(depth_map, cv2.COLORMAP_MAGMA)

        cv2.putText(img, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
        cv2.putText(depth_map, f'Depth Map Test 5.2', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
        cv2.imshow('Image', img)
        cv2.imshow('Depth Map', depth_map)

    key = cv2.waitKey(1)    
    if key == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()
